<a href="https://colab.research.google.com/github/polina-minaeva/income_prediction/blob/main/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%9B%D0%BE%D0%B3%D0%B8%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%81%D0%B8%D1%8F_%D0%B8_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Загрузка данных

In [ ]:
#Импортировала пандас для работы с данными в виде таблицы
import pandas as pd

In [ ]:
people = pd.read_csv('adult.csv')
people.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [ ]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


2. Предобраборка данных

In [ ]:
#Удалила пропуски в данных. Мы точно видим их в workclass и occupation, и это – знак "?"
people_onlydata = people[~(people['occupation'] == "?")]
people_onlydata = people_onlydata[~(people_onlydata['workclass'] == "?")]
people_onlydata.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


In [ ]:
#Проверила, остались ли пропуски
people_onlydata.loc[people_onlydata['education'] == "?"]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income


In [ ]:
people_onlydata.loc[people_onlydata['capital-gain'] == "?"]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income


In [ ]:
people_onlydata.loc[people_onlydata['hours-per-week'] == "?"]

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income


In [ ]:
#Проверила, влияет ли признак education на зарплату, с помощью графика

data_for_plot = people_onlydata[['income', 'education']]
data_for_plot.head()

,income,education
0,<=50K,11th
1,<=50K,HS-grad
2,>50K,Assoc-acdm
3,>50K,Some-college
5,<=50K,10th


In [ ]:
import numpy as np
from google.colab import autoviz

def heatmap(df, x_colname, y_colname, figscale=1, mpl_palette_name='viridis'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  import pandas as pd
  plt.subplots(figsize=(8 * figscale, 8 * figscale))
  df_2dhist = pd.DataFrame({
      x_label: grp[y_colname].value_counts()
      for x_label, grp in df.groupby(x_colname)
  })
  sns.heatmap(df_2dhist, cmap=mpl_palette_name)
  plt.xlabel(x_colname)
  plt.ylabel(y_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = heatmap(_df_11, *['income', 'education'], **{})
chart

In [ ]:
#На графике видно, что у людей с разными образованием отличается зарплата

3. Преообразование признаков и целевой переменной

In [ ]:
#Преобразовала категориальные признаки workclass, education, occupation с помощью one-hot encoding

In [ ]:
#Выбрала признаки для X
x_features = people_onlydata[['workclass', 'education', 'capital-gain', 'hours-per-week']]

In [ ]:
X = pd.get_dummies(x_features, columns=['workclass', 'education'])

In [ ]:
X

,capital-gain,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,...,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college
0,0,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,50,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,40,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,7688,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,30,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,0,38,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
48838,0,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
48839,0,40,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
48840,0,20,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
#Преобразовала целевую переменную из категориального типа в числовой с поомощью LabelEncoder

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(people_onlydata['income'])

LabelEncoder()

In [ ]:
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [ ]:
le.transform(['<=50K', '>50K'])

array([0, 1])

In [ ]:
y = pd.Series(data=le.transform(people_onlydata['income']))
y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

4. Построение модели логистической регрессии

In [ ]:
#Для начала попробовала использовать логистическую регрессию
#Загрузила нужные библиотеки для построения модели, использовала pipeline для одновременного запуска двух моделей
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

model = make_pipeline(
    StandardScaler(),
    LogisticRegression(max_iter=1000)
)

In [ ]:
#Разделила выборку на 20% теста, 80% тренировки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#Спрогнозировала данные
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
predictions[:5]

array([0, 0, 1, 1, 0])

In [ ]:
#Определила вероятность принадлежности к классу через predict_proba
model.predict_proba(X_test)

array([[0.66923424, 0.33076576],
       [0.68237476, 0.31762524],
       [0.36843503, 0.63156497],
       ...,
       [0.92516338, 0.07483662],
       [0.96815924, 0.03184076],
       [0.89712317, 0.10287683]])

In [ ]:
#Проверила точность модели
model.score(X_train, y_train)

0.8038342475424971

In [ ]:
model.score(X_test,y_test)

0.799174541110025

5. Построение модели SVM

In [ ]:
#Попробовала использовать метод SVM
from sklearn.svm import SVC

clf = make_pipeline(StandardScaler(), SVC())
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('svc', SVC())])

In [ ]:
clf.score(X_train, y_train)

0.8063596372128388

In [ ]:
clf.score(X_test, y_test)

0.8019984794178343

6. Выводы

Для построения модели потребовалась преобработка данных. Нужно было убрать пропуски, преобразовать категориальные переменные в числовые и также стандартизировать данные.

В рамках работы потребовалось построить две модели. SVC справилась лучше с токи зрения точности предсказания, однако заняла больше времени.

80% успешных предсказаний – неплохой результат, который тем не менее можно улучшить. Например, добавить в X другие признаки - попробовать разные варианты.